In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')

In [ ]:
pass_id = test_df['PassengerId']

# Invastigate the Data

In [ ]:
print(train_df.info())
print('---------------')
print(test_df.info())

- Age, Cabin, Fare have nulls in test 
- Age, Cabin, Embarked have nulls in train


In [ ]:
#checking nan values percentage
total = train_df.isnull().sum().sort_values(ascending=False)
percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

# Is the ticket related to class or sibiling 

In [ ]:
train_df['Ticket'].value_counts()

In [ ]:
train_df[train_df['Ticket'] == '347082']

# from the name we can figure that every family have the same ticket number 
# and most likely will have same cabin and this could help us fill the cabin feature

In [ ]:
train_df.describe()

- 38 % survived 
- most of passengers are youth (30 years old)


In [ ]:
train_df.describe(include=['O'])

In [ ]:
male_perc = 577/891
male_perc

- So male are 64% of the passengers
- it seems that Cabin have duplicates, indicates that every family stay in the same cabin 
- also Ticket have duplicates as we see that family have same Ticket number 

I want to know how well does each feature correlate with Survival. We want to do this early in our project and match these quick correlations with modelled correlations later in the project.

We may also add to our assumptions based on the problem description noted earlier.

- Women (Sex=female) were more likely to have survived.
- Children (Age<2) were more likely to have survived.
- The upper-class passengers (Pclass=1) were more likely to have survived.

In [ ]:
# I'll make sure of the competition assumption
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
# I'll make sure of the competition assumption
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
# I'll make sure of the competition assumption
train_df[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# Logically, the more you have a siblings the less probabilty of surviving  

In [ ]:
# I'll make sure of the competition assumption
train_df[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
sns.distplot(train_df['Age'])

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

# Children less than 2 years old have higher survival rate more than anyone

- I want to see if the fare related to survival rate 


In [ ]:
corr = train_df.select_dtypes(exclude=['object']).corr()
abs(corr['Survived']).sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(corr, annot = True)
plt.show()

# So there's no correlation between numerical variables -- and the most correlated feature with survival were the Pclass

# Let's Invastigate the Categorical features


- I want to figure out wether specific embarked have specific class or all embarked ar equal 

In [ ]:
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot(kind='bar')

In [ ]:
plt.bar(data = train_df, height = 'Fare',x = 'Survived')

# Observations.

- Higher fare paying passengers had better survival. 
- Embarked C have higher survival rate which maybe contain more from Class 1

# Now I want to complete, drop, engineer my data based on the EDA

In [ ]:
train_df.isnull().sum()

# I'll drop PassengerId and Cabin and Ticket featuers as i belive that it'll not help

In [ ]:
train_dff = train_df.drop(columns = ['Cabin', 'Ticket', 'PassengerId']) 
test_dff = test_df.drop(columns = ['Cabin', 'Ticket', 'PassengerId']) 

In [ ]:
train_dff['Sex'] = train_dff['Sex'].replace({'female':1,'male':0})
test_dff['Sex'] = test_dff['Sex'].replace({'female':1,'male':0})

In [ ]:
train_dff[train_dff['Age'] < 16]

In [ ]:
combine = [train_dff, test_dff]

In [ ]:
train_dff[train_dff['Age'].isnull()]

In [ ]:
nulls = train_dff[train_dff['Age'].isnull()]
nulls2 = test_dff[test_dff['Age'].isnull()]
nulls = list(nulls['Name'])
nulls2 = list(nulls2['Name'])

In [ ]:
# Fill Age in train data with 2 several random ratios, if the person survived it will get age from 22 to 36 
# and if he survived he will get age from 37 to 60
for i, c in enumerate(train_dff['Survived']):
    if train_dff.loc[i,'Name'] in nulls: 
        if c == 0:
            train_dff.loc[i,'Age'] =  np.random.randint(22, 36) # Youth
        else:
            train_dff.loc[i,'Age'] =  np.random.randint(37, 60) # Elder

In [ ]:
for i, c in enumerate(test_dff['Name']):
    if c in nulls2:
        test_dff.loc[i,'Age'] =  np.random.randint(16, 50)

In [ ]:
print(train_dff.Age.isnull().sum())
print(test_dff.Age.isnull().sum())

# let's create Age bands then convert age feature to [0, 1, 2, 3, 4]

In [ ]:
# This code isn't important i was just practice how may i make bands when i need 

train_dff['AgeBand'] = pd.cut(train_dff['Age'], 5)
train_dff[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)


In [ ]:
train_dff[train_dff['Age'] < 16]

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
train_dff.head()

In [ ]:
train_dff = train_dff.drop(['AgeBand'], axis=1)
combine = [train_dff, test_dff]
train_dff.head()

In [ ]:
train_dff['Age'].unique()
test_dff['Age'].unique()

# I'll create family feature which is combination between sibling and parch

In [ ]:
for dataset in combine:
    dataset['Family'] = dataset['SibSp'] + dataset['Parch'] 

In [ ]:
train_dff[['Family', 'Survived']].groupby(['Family'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df = train_dff.drop(['Parch', 'SibSp'], axis=1)
test_df = test_dff.drop(['Parch', 'SibSp'], axis=1)
combine = [train_df, test_df]

train_df.head()

In [ ]:
train_df['Fare'].describe()

In [ ]:
sns.distplot(train_df['Fare'])

# I'll make a flag feature return 1 if the passenger is alone

In [ ]:
for dataset in combine:
    dataset['Alone'] = 0
    dataset.loc[train_df['Family'] == 0,'Alone'] = 1

In [ ]:
train_df['Embarked'].isnull().sum()

In [ ]:
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

In [ ]:
sns.distplot(train_df['Fare'])

In [ ]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

# I may convert it to intervals also because a lot of people pay in a range 0 to 150 

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

combine = [train_df, test_df]
    
train_df.head(10)

In [ ]:
'''for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)'''

In [ ]:
lr = LabelEncoder()
train_df['Embarked'] = lr.fit_transform(train_df['Embarked'])
train_df

In [ ]:
test_df['Embarked'] = lr.transform(test_df['Embarked'])

In [ ]:
train_df = train_df.drop('Name', axis = 1)
test_df = test_df.drop('Name', axis = 1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.model_selection import GridSearchCV

In [ ]:
X = train_df.drop('Survived', axis =1)
y = train_df['Survived']
print(X.shape, test_df.shape)

In [ ]:
X.head()

In [ ]:
'''X['Fare'] = np.log(X['Fare'])
test_df['Fare'] = np.log(test_df['Fare'])'''

In [ ]:
X.head()

In [ ]:
'''OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X[['Embarked']]))

OH_cols_test = pd.DataFrame(OH_encoder.transform(test_df[['Embarked']]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X.index
OH_cols_test.index = test_df.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X.drop('Embarked', axis=1)
num_X_test = test_df.drop('Embarked', axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)'''

In [ ]:
Skwed_features = ['Fare'] # need log
##cat_features = ['Embarked'] # need encoding

In [ ]:
preprocessor = ColumnTransformer([
    ('log_trans', FunctionTransformer(func = np.log1p, validate = False), Skwed_features)#,
    ##('label',OneHotEncoder(), cat_features)
])

In [ ]:
'''X = preprocessor.fit_transform(X)
test_df = preprocessor.transform(test_df)'''

In [ ]:
model = LogisticRegression()
param_grid = {'penalty' : ['l2'],
             'C':[0.1,0.5,0.001,0.005]}
grid_cv = GridSearchCV(model, param_grid, cv=5, scoring = 'accuracy')
grid_cv.fit(X,y)
grid_cv.score(X,y)
Y_pred = grid_cv.predict(test_df)

In [ ]:
model = KNeighborsClassifier(n_neighbors=5)
param_grid = {'n_neighbors':[3, 4, 5]}
grid_cv = GridSearchCV(model, param_grid, cv=5,  scoring = 'accuracy')
grid_cv.fit(X,y)
grid_cv.score(X,y)

In [ ]:
model = RandomForestClassifier()
param_grid = {'n_estimators': [25,50, 100],
             'max_depth':[2,4,8,10,12]}
grid_cv = GridSearchCV(model, param_grid, cv=5,  scoring = 'accuracy')
grid_cv.fit(X,y)
Y_pred2 = grid_cv.predict(test_df)
grid_cv.score(X,y)

In [ ]:
grid_cv.best_params_

In [ ]:
'''from tensorflow import keras
from tensorflow.keras import layers

# YOUR CODE HERE: define the model given in the diagram
model = keras.Sequential([
    layers.BatchNormalization(input_shape=[X.shape[1]]),
    layers.Dense(units = 256),
    layers.BatchNormalization(),
    layers.Dropout(rate=0.3),
    layers.Dense(units = 256),
    layers.BatchNormalization(),
    layers.Dropout(rate=0.3),
    layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer = 'adam',
              loss ='binary_crossentropy',
             metrics = ['binary_accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

model.fit(
    X, y,
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping],
)'''

In [ ]:
#Y_pred = model.predict(test_df)
#Y_pred = np.round(Y_pred.reshape(418,))

In [ ]:
Y_pred = Y_pred.astype(int) 

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(X, y)
svc.score(X, y)
#Y_pred = svc.predict(test_df)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": pass_id,
        "Survived": Y_pred2
    })
submission.to_csv('submission.csv', index = False)